# Stock Sentiment Analysis using News Headlines (CountVectorizer + Logistic)

<p><b>Subject</b>: AML<br>
<b>Rollno</b> : 20MCED05 & 20MCED08</p>
<b>Subject Code</b>: 3CS1111<br>


First things first, let's import the libraries we'll be using.  

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

[Pandas](http://pandas.pydata.org/) will make our data easy to look at and work with.  
[CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), part of scikit-learn, will take care of our NLP tasks.  
[LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), also part of scikit-learn, will train and test our predictive models.

----------

# Data Import

In [5]:
data = pd.read_csv('news.csv')

In [6]:
data.head(1)

Date  Label                                               Top1  \
0  2008-08-08      0  b"Georgia 'downs two Russian warplanes' as cou...   

                                      Top2  \
0  b'BREAKING: Musharraf to be impeached.'   

                                                Top3  \
0  b'Russia Today: Columns of troops roll into So...   

                                                Top4  \
0  b'Russian tanks are moving towards the capital...   

                                                Top5  \
0  b"Afghan children raped with 'impunity,' U.N. ...   

                                                Top6  \
0  b'150 Russian tanks have entered South Ossetia...   

                                                Top7  \
0  b"Breaking: Georgia invades South Ossetia, Rus...   

                                                Top8  ...  \
0  b"The 'enemy combatent' trials are nothing but...  ...   

                                               Top16  \
0  b'Georgia Invades South Ossetia - if Russia ge...   

                                 Top17  \
0  b'Al-Qaeda Faces Islamist Backlash'   

                                               Top18  \
0  b'Condoleezza Rice: "The US would not act to p...   

                                               Top19  \
0  b'This is a busy day:  The European Union has ...   

                                               Top20  \
0  b"Georgia will withdraw 1,000 soldiers from Ir...   

                                               Top21  \
0  b'Why the Pentagon Thinks Attacking Iran is a ...   

                                               Top22  \
0  b'Caucasus in crisis: Georgia invades South Os...   

                                               Top23  \
0  b'Indian shoe manufactory  - And again in a se...   

                                               Top24  \
0  b'Visitors Suffering from Mental Illnesses Ban...   

                                      Top25  
0  b"No Help for Mexico's Kidnapping Surge"  

[1 rows x 27 columns]

In [7]:
data.tail(1)

Date  Label                                               Top1  \
1988  2016-07-01      1  A 117-year-old woman in Mexico City finally re...   

                                                  Top2  \
1988  IMF chief backs Athens as permanent Olympic host   

                                                   Top3  \
1988  The president of France says if Brexit won, so...   

                                                   Top4  \
1988  British Man Who Must Give Police 24 Hours' Not...   

                                                   Top5  \
1988  100+ Nobel laureates urge Greenpeace to stop o...   

                                                   Top6  \
1988  Brazil: Huge spike in number of police killing...   

                                                   Top7  \
1988  Austria's highest court annuls presidential el...   

                                                   Top8  ...  \
1988  Facebook wins privacy case, can track any Belg...  ...   

                                                  Top16  \
1988  The United States has placed Myanmar, Uzbekist...   

                                                  Top17  \
1988  S&amp;P revises European Union credit rating t...   

                                                  Top18  \
1988  India gets $1 billion loan from World Bank for...   

                                                  Top19  \
1988  U.S. sailors detained by Iran spoke too much u...   

                                                  Top20  \
1988  Mass fish kill in Vietnam solved as Taiwan ste...   

                                                  Top21  \
1988  Philippines president Rodrigo Duterte urges pe...   

                                                  Top22  \
1988  Spain arrests three Pakistanis accused of prom...   

                                                  Top23  \
1988  Venezuela, where anger over food shortages is ...   

                                                  Top24  \
1988  A Hindu temple worker has been killed by three...   

                                                  Top25  
1988  Ozone layer hole seems to be healing - US &amp...  

[1 rows x 27 columns]

Next, let's take a look at the data with the [head](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) method.

In [8]:
data.shape

(1989, 27)

We've got a lot of vaiables here, but the layout is pretty straight-forward.  
As a reminder, the Label variable will be a **1** if the DJIA **stayed the same or rose** on that date or 
 **0** if the DJIA **fell** on that date.

In [9]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

----------

# Text Preprocessing

Now that our data is loaded in, we need to clean it up just a little bit to prepare it for the rest of our analysis.  
To illustrate this process, look at how the example headline below changes from cell to cell.  
Don't worry about the code too much here, since this example is only meant to be visual.

In [10]:
train.tail()

Date  Label                                               Top1  \
1606  2014-12-24      1  Death toll among Qatars 2022 World Cup workers...   
1607  2014-12-26      1  Saudis are eagerly awaiting the approval of a ...   
1608  2014-12-29      0  Solar Power Storage Prices Drop 25% In Germany...   
1609  2014-12-30      0  China businessman jailed for 13 years for buyi...   
1610  2014-12-31      0  AirAsia flight found at the bottom of the Java...   

                                                   Top2  \
1606  Fishing Supertrawlers to be banned permanently...   
1607  Due to the fall in oil prices, Saudi Arabia is...   
1608  North Korea Hit Again By Internet Outage; Expe...   
1609  AirAsia live: Emergency slide, plane door seen...   
1610  North Korean defector details 'human experimen...   

                                                   Top3  \
1606  Indian telecommunications company Airtel viola...   
1607  Bill giving government the power to shutdown t...   
1608  ARCHAEOLOGY - Massive ancient underground city...   
1609  AirAsia plane wreckage found, bodies being rec...   
1610  Korean Air ex-executive Cho Hyun-ah arrested -...   

                                                   Top4  \
1606  North Korea's Internet is down again; second b...   
1607  A struggle for women's rights is brewing withi...   
1608  Reopen investigation into Westminster pedophil...   
1609  Scotland confirms case of Ebola - Ebola cases ...   
1610  South Korean to drop Sony film "The Interview"...   

                                                   Top5  \
1606  Jakarta to ban virginity tests for female civi...   
1607  Putin cancels New Year's Holiday for governmen...   
1608  Taliban declare 'defeat' of U.S., allies in Af...   
1609  Pope Francis to Catholics: It's time to take a...   
1610  U.S. opening of oil export widens battle: The ...   

                                                   Top6  \
1606  Workers at Chinese iPhone factories are pressu...   
1607  Ireland is siding with Microsoft in its legal ...   
1608  British Falklands defence review after militar...   
1609  Oil hits five-and-a-half-year low below $57 on...   
1610  FBI agents investigating the Sony Pictures hac...   

                                                   Top7  \
1606  Thousands in Spain protest ban on demonstratio...   
1607  More than 80 people including women and childr...   
1608    Virgin Confirms Fault On Plane Circling Over UK   
1609  "Lizard Squad" members who ruined Christmas fo...   
1610  Indian Government Orders Blocking of 32 Websit...   

                                                   Top8  ...  \
1606  London activists occupy 5-storey former bank i...  ...   
1607  The Syrian Army has killed nearly 5,000 Islami...  ...   
1608  Photo of 'women murdered for refusing sex jiha...  ...   
1609  Two students in Thailand face up to 15 years i...  ...   
1610  Russia annual inflation jumps to 11.4% as roub...  ...   

                                                  Top16  \
1606  Scientists discover oldest stone tool ever fou...   
1607  Senior UN officials expelled from Sudan follow...   
1608  Islamic State executed nearly 2,000 people in ...   
1609  Turkey is being used as one of the primary rou...   
1610  Newly published NSA documents show agency coul...   

                                                  Top17  \
1606  ISIS claims capture of Jordanian pilot after p...   
1607  Mauritania journalist sentenced to death for a...   
1608  The leader of one of Mexico's first anti-crime...   
1609  Swedish mosque set ablaze in second suspected ...   
1610  Britain says it cannot support new Palestinian...   

                                                  Top18  \
1606  Russian prime minister warns of 'deep recessio...   
1607  Kim Dotcom Makes Deal with LizardSquad to stop...   
1608                    Gmail has been blocked in China   
1609  Cybersecurity Firm Identifies Six In Sony Hack...   
1610      Bolivias Mora

In [11]:
example = train.iloc[3,10]
print(example)

b"The commander of a Navy air reconnaissance squadron that provides the President and the defense secretary the airborne ability to command the nation's nuclear weapons has been relieved of duty"


In [12]:
example2 = example.lower()
print(example2)

b"the commander of a navy air reconnaissance squadron that provides the president and the defense secretary the airborne ability to command the nation's nuclear weapons has been relieved of duty"


In [13]:
example3 = CountVectorizer().build_tokenizer()(example2)
print(example3)

['the', 'commander', 'of', 'navy', 'air', 'reconnaissance', 'squadron', 'that', 'provides', 'the', 'president', 'and', 'the', 'defense', 'secretary', 'the', 'airborne', 'ability', 'to', 'command', 'the', 'nation', 'nuclear', 'weapons', 'has', 'been', 'relieved', 'of', 'duty']


In [14]:
pd.DataFrame([[x,example3.count(x)] for x in set(example3)], columns = ['Word', 'Count'])

Word  Count
0          ability      1
1          nuclear      1
2             that      1
3        president      1
4         provides      1
5          command      1
6           nation      1
7   reconnaissance      1
8               to      1
9             duty      1
10            been      1
11       commander      1
12        relieved      1
13             and      1
14        squadron      1
15             has      1
16         weapons      1
17             air      1
18       secretary      1
19        airborne      1
20             the      5
21            navy      1
22              of      2
23         defense      1

Were you able to see everything that changed?  
The process involved:  
- Converting the headline to lowercase letters  
- Splitting the sentence into a list of words  
- Removing punctuation and meaningless words  
- Transforming that list into a table of counts

----------

# Basic Model Training and Testing

As mentioned previously, scikit-learn is going to take care of all of our preprocessing needs.  
The tool we'll be using is CountVectorizer, which takes a single list of strings as input, and produces word counts for each one.

You might be wondering if our dataframe meets this "single list of strings" criteria, and the answer to that is... it doesn't!  
In order to meet this criteria, we'll use the following [for loop](https://wiki.python.org/moin/ForLoop) to iterate through each row of our dataset, [combine](https://docs.python.org/3.5/library/stdtypes.html#str.join) all of our headlines into a single string, then [add](https://docs.python.org/3.5/tutorial/datastructures.html) that string to the list we need for CountVectorizer.

In [15]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [16]:
trainheadlines

['b"Georgia \'downs two Russian warplanes\' as countries move to brink of war" b\'BREAKING: Musharraf to be impeached.\' b\'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)\' b\'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire\' b"Afghan children raped with \'impunity,\' U.N. official says - this is sick, a three year old was raped and they do nothing" b\'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.\' b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\'s side" b"The \'enemy combatent\' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." b\'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]\' b\'Did the U.S. Prep Georgia for War with Russia?\

With our headlines formatted, we can set up our CountVectorizer.  
To start, let's just use the default settings and see how it goes!  
Below, we'll name our default vectorizer, then [use](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit_transform) it on our list of combined headlines.  
After that, we'll take a look at the size of the result to see how many words we have.

In [17]:
train.shape

(1611, 27)

In [18]:
corpus = [
     'This is the first document',
     'This document is the second document',
     ]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.vocabulary_)
print(X)
print(X.shape)

{'this': 5, 'is': 2, 'the': 4, 'first': 1, 'document': 0, 'second': 3}
  (0, 5)	1
  (0, 2)	1
  (0, 4)	1
  (0, 1)	1
  (0, 0)	1
  (1, 5)	1
  (1, 2)	1
  (1, 4)	1
  (1, 0)	2
  (1, 3)	1
(2, 6)


In [20]:
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

(1611, 31675)


In [21]:
print(basictrain)

  (0, 12120)	10
  (0, 9116)	1
  (0, 29313)	2
  (0, 24572)	5
  (0, 30656)	1
  (0, 2705)	1
  (0, 7138)	2
  (0, 18619)	1
  (0, 28628)	7
  (0, 4631)	1
  (0, 20034)	11
  (0, 30614)	5
  (0, 4542)	2
  (0, 18776)	1
  (0, 3561)	2
  (0, 14201)	1
  (0, 24571)	5
  (0, 28636)	2
  (0, 6326)	1
  (0, 29118)	2
  (0, 24352)	1
  (0, 14898)	1
  (0, 26517)	7
  (0, 20360)	7
  (0, 11399)	1
  :	:
  (1610, 17862)	2
  (1610, 15268)	1
  (1610, 19682)	1
  (1610, 5581)	2
  (1610, 16817)	1
  (1610, 21809)	1
  (1610, 2566)	1
  (1610, 7953)	1
  (1610, 2493)	1
  (1610, 12953)	1
  (1610, 25304)	1
  (1610, 25456)	1
  (1610, 19618)	1
  (1610, 4605)	1
  (1610, 1810)	1
  (1610, 19943)	1
  (1610, 17897)	1
  (1610, 6935)	1
  (1610, 24430)	1
  (1610, 5813)	1
  (1610, 22065)	1
  (1610, 12230)	1
  (1610, 26511)	1
  (1610, 8195)	1
  (1610, 6235)	1


Wow! Our resulting table contains counts for 31,675 different words!

Now, let's train a logistic regression model using this data.  
In the cell below, we're simply naming our model, then [fitting](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.fit) the model based on our X and Y values.

In [22]:
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"])

D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Our model is ready to go, so let's set up our test data.  
Here, we're just going to repeat the steps we used to prep our training data, then [predict](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.predict) whether the DJIA increased or decreased for each day in the test dataset.

In [23]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
basictest = basicvectorizer.transform(testheadlines)
predictions = basicmodel.predict(basictest)

The predictions are set, so let's use a [crosstab](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.crosstab.html) to take a look at the results!

In [24]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
matrix=confusion_matrix(test['Label'],predictions)
print(matrix)
score=accuracy_score(test['Label'],predictions)
print(score)
report=classification_report(test['Label'],predictions)
print(report)


[[ 61 125]
 [ 92 100]]
0.42592592592592593
              precision    recall  f1-score   support

           0       0.40      0.33      0.36       186
           1       0.44      0.52      0.48       192

    accuracy                           0.43       378
   macro avg       0.42      0.42      0.42       378
weighted avg       0.42      0.43      0.42       378



Prediction accuracy is just over 42%. It seems like this model isn't too reliable.  
Now, let's also take a look at the coefficients of our model. 

The cell below will get a list of the names from our CountVectorizer and a list of the coefficients from our model, then combine the two lists into a Pandas dataframe.  
Once that's made, we can sort it and check out the top 10 positive and negative coefficients.

----------